In [1]:
from pyzbar.pyzbar import decode
from pdf2image import convert_from_path
import os
import numpy as np

In [3]:
# aqui buscamos dentro do caminho somente os arquivos .pdf, e no caso so permitira dentro da lista arquivos que sejam .pdf
pdfs = [i for i in os.listdir() if '.pdf' in i]
pdf_path = pdfs[1]
display(pdf_path)

'DAS-PAGO.pdf'

In [4]:
# aqui vamos converter um arquivo pdf em formato de imagem
pages = convert_from_path(pdf_path)
img = pages[0]

In [29]:
# decodificando foto e pegando os dados
# podemos verificar o tipo do código de barras, usaremos .type
def detectando_cod_bars():
    detected_datas = decode(img)
    if not detected_datas:
        return False
    else:
        for barcode in detected_datas:
            barcode = detected_datas[1]
            if barcode.data != '' and barcode.type == 'I25':
                # retornar a uma string do python
                barcode_ = barcode.data.decode('utf-8')
    return barcode_


class linha_digitavel():

    def tipo_boleto(barcode):
        if barcode[0] == '8':
            match barcode[1]:
                case '1':
                    print('Prefeituras')
                case '2':
                    print('Saneamento')
                case '3':
                    print('Energia Elétrica e Gás')
                case '4':
                    print('Telecomunicação')
                case '5':
                    print('Órgãos Governamentais')
                case '6':
                    print(
                        'Carnês e assemelhados ou demais empresas/orgãos id através do CNPJ')
                case '7':
                    print('Multas de trânsito')
                case '9':
                    print('Uso exclusivo do banco')
            match barcode[2]:
                case '6':
                    print('Modulo 10')
                case '7':
                    print('Modulo 10')
                case '8':
                    print('Modulo 11')
                case '9':
                    print('Modulo 11')

    def modulo11(barcode):
        pesos = [4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]

        segmentos = []
        start = 0
        digitos = []

        for i in range(4):
            segmentos.append(barcode[start:start+11])
            start += 11

        for segmento in segmentos:
            calculo_valores = [int(campo) * peso for campo,
                               peso in zip(segmento, pesos)]
            soma_campos = sum(calculo_valores)
            soma_campos = soma_campos % 11
            if soma_campos in [0, 1]:
                soma_campos = 0
            elif soma_campos == 10:
                soma_campos = 1
            else:
                soma_campos = 11 - soma_campos

            digitos.append(soma_campos)
        return digitos


barcode = '85850000000656003282232507082228658638091205'
print(linha_digitavel.modulo11(barcode))

[2, 0, 1, 7]
